**Oбучение с подкреплением**

Евгений Борисов borisov.e@solarl.ru

In [1]:
import numpy as np
import gym

In [2]:
env = gym.make('CartPole-v1')
# ENV_NAME = 'BreakoutDeterministic-v4'

In [3]:
s = env.reset()
env.render()
for t in range(150):
    a = env.action_space.sample()
    sn,r,d,_ = env.step(a)
    env.render()
    print('%d: %.1f %d'%(t,r,d) )
    s=sn
    #if d: break
env.close()  

0: 1.0 0
1: 1.0 0
2: 1.0 0
3: 1.0 0
4: 1.0 0
5: 1.0 0
6: 1.0 0
7: 1.0 0
8: 1.0 0
9: 1.0 0
10: 1.0 0
11: 1.0 0
12: 1.0 0
13: 1.0 0
14: 1.0 1
15: 0.0 1
16: 0.0 1
17: 0.0 1
18: 0.0 1
19: 0.0 1
20: 0.0 1
21: 0.0 1
22: 0.0 1
23: 0.0 1
24: 0.0 1
25: 0.0 1


/usr/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


26: 0.0 1
27: 0.0 1
28: 0.0 1
29: 0.0 1
30: 0.0 1
31: 0.0 1
32: 0.0 1
33: 0.0 1
34: 0.0 1
35: 0.0 1
36: 0.0 1
37: 0.0 1
38: 0.0 1
39: 0.0 1
40: 0.0 1
41: 0.0 1
42: 0.0 1
43: 0.0 1
44: 0.0 1
45: 0.0 1
46: 0.0 1
47: 0.0 1
48: 0.0 1
49: 0.0 1
50: 0.0 1
51: 0.0 1
52: 0.0 1
53: 0.0 1
54: 0.0 1
55: 0.0 1
56: 0.0 1
57: 0.0 1
58: 0.0 1
59: 0.0 1
60: 0.0 1
61: 0.0 1
62: 0.0 1
63: 0.0 1
64: 0.0 1
65: 0.0 1
66: 0.0 1
67: 0.0 1
68: 0.0 1
69: 0.0 1
70: 0.0 1
71: 0.0 1
72: 0.0 1
73: 0.0 1
74: 0.0 1
75: 0.0 1
76: 0.0 1
77: 0.0 1
78: 0.0 1
79: 0.0 1
80: 0.0 1
81: 0.0 1
82: 0.0 1
83: 0.0 1
84: 0.0 1
85: 0.0 1
86: 0.0 1
87: 0.0 1
88: 0.0 1
89: 0.0 1
90: 0.0 1
91: 0.0 1
92: 0.0 1
93: 0.0 1
94: 0.0 1
95: 0.0 1
96: 0.0 1
97: 0.0 1
98: 0.0 1
99: 0.0 1
100: 0.0 1
101: 0.0 1
102: 0.0 1
103: 0.0 1
104: 0.0 1
105: 0.0 1
106: 0.0 1
107: 0.0 1
108: 0.0 1
109: 0.0 1
110: 0.0 1
111: 0.0 1
112: 0.0 1
113: 0.0 1
114: 0.0 1
115: 0.0 1
116: 0.0 1
117: 0.0 1
118: 0.0 1
119: 0.0 1
120: 0.0 1
121: 0.0 1
122: 0.0 1
123: 0.

---

In [4]:
from keras.models import Sequential
from keras.layers import Dense
# from keras.layers import Dropout
# from keras.optimizers import Adam
from keras.optimizers import SGD

Using TensorFlow backend.


In [5]:
def create_model(env):
    md = Sequential()
    md.add(Dense(256, activation='relu',input_shape=env.observation_space.shape))
    md.add(Dense(256, activation='relu'))
#    md.add(Dense(32, activation='relu'))
    md.add(Dense(env.action_space.n))
    md.compile(loss='mean_squared_error', optimizer=SGD(lr= 0.001))
    return md

In [6]:
model = create_model(env)
model_target = create_model(env)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
HISTORY_DEEP = 2000 # глубина истории переходов
BATCH_SIZE = 32 # размер батча для обучения модели

GAMMA = 0.85 # коэффициент для расчёта оценки reward
TAU = 0.125 # определяет различие весов основной и таргет моделей

# вероятность использования случайного действия при обучении
EPSILON_MAX = 1.0 # в начальном состоянии используем случайный выбор действия
EPSILON_MIN = 0.05 # когда модель начала обучаться позволяем ей оценивать действия
EPSILON_DECAY = 0.995 # шаг изменения вероятности

MAX_EPOCH = 200 # максимальное количество эпох обучения
MAX_STEP = 500  # максимальное количество шагов в эпохе (длительность одной игры)


In [8]:
def reshape_state(s): return s[np.newaxis,:]

In [9]:
# выбор действия
def get_action(model,state,eps=EPSILON_MAX): # с вероятностью eps выбираем случайное действие
    if np.random.rand()<eps: return env.action_space.sample(), 1
    return np.argmax( model.predict( state )[0] ), 0

In [10]:
def replay(model, model_target, history, batch_size=BATCH_SIZE,gamma=GAMMA):

    if len(history) < batch_size: return
    
    samples = [ history[i] for i in np.random.permutation(len(history))[:batch_size] ] 

    for s in samples:
        target = model_target.predict( s['state0'] )
        
        target[0][ s['act'] ] = s['reward'] 
        
        if s['isdone']:
            target[0][ s['act'] ] = s['reward']
        else:
            target[0][ s['act'] ] = s['reward'] + gamma*max(model_target.predict(s['state1'])[0])
            
        # model.fit( s['state0'], target, epochs=1, verbose=0)
        model.train_on_batch(s['state0'], target)

In [11]:
def target_train(model, model_target,tau=TAU):
    weights = model.get_weights()
    target_weights = model_target.get_weights()
    for i in range(len(target_weights)):
        target_weights[i] = weights[i] * tau + target_weights[i] * (1. - tau)
    model_target.set_weights(target_weights)

In [12]:
%%time

history = []

epsilon = EPSILON_MAX # вероятность использования случайного действия при обучении

for e in range(MAX_EPOCH):
    # инициализируем среду
    state0 = env.reset() 
    state0 = reshape_state(state0)
    for st in range(MAX_STEP):
        
        # выбираем действие
        act, is_act_rand = get_action(model,state0,epsilon)
        # применяем действие
        state1, reward, done, _ = env.step(act)
        state1 = reshape_state(state1)
    
        # сохраняем историю
        history.append({'state0':state0,'state1':state1,'act':act,'reward':reward,'isdone':done})
        history = history[-HISTORY_DEEP:] # ограничиваем историю

        # коррекция основной модели
        replay(model, model_target, history)
        
        # коррекция таргет модели
        target_train(model, model_target)
                
        if done: break
        state0 = state1
        epsilon = max(EPSILON_MIN, epsilon*EPSILON_DECAY)
                
    print('%d/%d : %d steps'%(e+1,MAX_EPOCH,st+1))  
    
    if st>300: 
        print('STEP BOUND!')
        break

# history

Instructions for updating:
Use tf.cast instead.
1/200 : 48 steps
2/200 : 29 steps
3/200 : 64 steps
4/200 : 12 steps
5/200 : 10 steps
6/200 : 11 steps
7/200 : 18 steps
8/200 : 10 steps
9/200 : 8 steps
10/200 : 9 steps
11/200 : 13 steps
12/200 : 29 steps
13/200 : 15 steps
14/200 : 11 steps
15/200 : 11 steps
16/200 : 8 steps
17/200 : 13 steps
18/200 : 9 steps
19/200 : 13 steps
20/200 : 10 steps
21/200 : 8 steps
22/200 : 12 steps
23/200 : 14 steps
24/200 : 14 steps
25/200 : 51 steps
26/200 : 123 steps
27/200 : 99 steps
28/200 : 40 steps
29/200 : 119 steps
30/200 : 112 steps
31/200 : 188 steps
32/200 : 161 steps
33/200 : 81 steps
34/200 : 48 steps
35/200 : 60 steps
36/200 : 43 steps
37/200 : 110 steps
38/200 : 178 steps
39/200 : 139 steps
40/200 : 197 steps
41/200 : 211 steps
42/200 : 211 steps
43/200 : 500 steps
STEP BOUND!


In [22]:
# len(history)

In [20]:
# st=300
model.save('CartPole-%d.model'%(st))

---

In [ ]:
# from keras.models import load_model 
# model = load_model('CartPole-288.model')

In [14]:
s = env.reset()[np.newaxis,:]
env.render()
#while True:
for t in range(350):    
    a = np.argmax( model.predict(s)[0] )   
    s,r,d,_ = env.step(a)
    s=s[np.newaxis,:]
    env.render()
    if d: 
        print(t) 
        break

In [15]:
env.close()    